In [1]:
# functional demonstration of use of the two-layered neural network classifier 
# for classification on doc2vec docs vectors

# inspired by
# https://deeplearning4j.org/word2vec.html
# https://deeplearning4j.org/welldressed-recommendation-engine
# https://www.tensorflow.org/get_started/tflearn
# http://scikit-learn.org/stable/modules/cross_validation.html

# author Michal Stefanik  mistefan  at  redhat dot com

import numpy as np
import pandas as pd
import random

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.estimators.estimator import SKCompat

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from doc2vec_wrapper import D2VWrapper

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# optimization steps are reduced on TEST_MODE
TEST_MODE = False

In [3]:
# initialize d2v_wrapper providing as well metadata about the models state
d2v_wrapper = D2VWrapper(content_basepath="../../data/content/playground",
                         basepath_suffix="_content.csv",
                         content_categories=["eap", "fuse", "devstudio"],
                         vector_length=500)

# initialize the vocab of documents and minimize the distances of embeddings in training
d2v_wrapper.init_model_vocab()
d2v_wrapper.train_model(shuffle=True, epochs=1 if TEST_MODE else 10)

# transform the doc data set into a vector set
doc_vectors_labeled = d2v_wrapper.infer_content_vectors()
doc_vectors = doc_vectors_labeled.iloc[:, :-1]
doc_labels = doc_vectors_labeled.iloc[:, -1]

2017-03-12 12:07:53,082 : INFO : collecting all words and their counts
2017-03-12 12:07:53,083 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-03-12 12:07:53,252 : INFO : PROGRESS: at example #10000, processed 1128886 words (6717319/s), 41680 word types, 10000 tags
2017-03-12 12:07:53,295 : INFO : PROGRESS: at example #20000, processed 1270990 words (3311594/s), 45697 word types, 20000 tags
2017-03-12 12:07:53,477 : INFO : PROGRESS: at example #30000, processed 2444612 words (6488107/s), 86971 word types, 30000 tags
2017-03-12 12:07:53,551 : INFO : collected 101367 word types and 33847 unique tags from a corpus of 33847 examples and 2895071 words
2017-03-12 12:07:53,765 : INFO : min_count=5 retains 18141 unique words (drops 83226)
2017-03-12 12:07:53,765 : INFO : min_count leaves 2766261 word corpus (95% of original 2895071)
2017-03-12 12:07:53,804 : INFO : deleting the raw counts dictionary of 101367 items
2017-03-12 12:07:53,808 : INFO : sample=0 

In [4]:
# extended evaluation metric on selected category
def accuracy_for_category(y_expected, y_actual, label):
    label_expected = y_expected[y_expected == label]
    intersect = y_expected[np.where(y_expected == y_actual)]
    label_intersect = intersect[intersect == label]

    return float(len(label_intersect)) / len(label_expected)

In [5]:
# encoding/decoding target categories
mapping = []


def encode_categories(target_series):
    global mapping
    if not len(mapping):
        for cat in target_series.unique():
            mapping.append(cat)

    return target_series.apply(lambda cat_str: mapping.index(cat_str))


def decode_categories(target_series):
    global mapping
    return target_series.apply(lambda cat_idx: mapping[cat_idx])


def dataset_from_dataframe(data_df, target_series):
    dataset = tf.contrib.learn.datasets.base.Dataset(data=data_df.values, target=encode_categories(target_series))
    return dataset

In [6]:
# classifier training and eval:
strat_kfold = StratifiedKFold(n_splits=1 if TEST_MODE else 5, shuffle=True)
accuracies = []
cat_accuracies = pd.DataFrame(columns=d2v_wrapper.content_categories)

for train_doc_indices, test_doc_indices in strat_kfold.split(doc_vectors, doc_labels):
    
    random.shuffle(train_doc_indices)
    random.shuffle(test_doc_indices)
    
    train_vectors = doc_vectors.iloc[train_doc_indices]

    y_true = doc_labels.iloc[train_doc_indices]

    tf_dataset = dataset_from_dataframe(train_vectors, y_true)
    
    # init model:
    # model aims to consist of two layers
    # 1. dense, fully-connected layer with relu act. function
    # 2. softmax output layer for classification and further relevance scoring
    feature_columns = [tf.contrib.layers.real_valued_column("", dimension=doc_vectors.shape[1])]

    two_layer_nn_classifier = SKCompat(tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                                                  hidden_units=[doc_vectors.shape[1] / 2],
                                                                  activation_fn=tf.nn.relu,
                                                                  dropout=0.05,
                                                                  n_classes=len(d2v_wrapper.content_categories),
                                                                  optimizer="Adam"))

    two_layer_nn_classifier.fit(x=tf_dataset.data,
                                y=tf_dataset.target,
                                steps=500 if TEST_MODE else 2000)

    # testing
    # can be used to infer the relative relevance (or score) of the document for categories    
    logits = two_layer_nn_classifier.predict(doc_vectors.iloc[test_doc_indices])
    
    y_actual = decode_categories(pd.Series(logits["classes"])).values
    y_expected = doc_labels.iloc[test_doc_indices].values

    # evaluation
    split_accuracy = accuracy_score(y_expected, y_actual)
    logging.warn("Run accuracy: %s" % split_accuracy)
    accuracies.append(split_accuracy)
    split_cat_accuracies = map(lambda cat: accuracy_for_category(y_expected, y_actual, cat),
                               d2v_wrapper.content_categories)
    logging.warn("Cat accuracies:\n%s" % split_cat_accuracies)
    cat_accuracies = cat_accuracies.append(
        pd.DataFrame(data=[split_cat_accuracies], columns=d2v_wrapper.content_categories))

2017-03-12 12:15:45,493 : WARNING : Using temporary folder as model directory: /tmp/tmpGq6CZN


INFO:tensorflow:Using default config.


2017-03-12 12:15:45,494 : INFO : Using default config.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d8c502850>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


2017-03-12 12:15:45,495 : INFO : Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d8c502850>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


...

INFO:tensorflow:Saving checkpoints for 2000 into /tmp/tmpUO4Emt/model.ckpt.


2017-03-12 12:17:00,200 : INFO : Saving checkpoints for 2000 into /tmp/tmpUO4Emt/model.ckpt.


INFO:tensorflow:Loss for final step: 0.231347.


2017-03-12 12:17:00,623 : INFO : Loss for final step: 0.231347.


2017-03-12 12:17:00,698 : WARNING : float64 is not supported by many models, consider casting to float32.
2017-03-12 12:17:00,945 : WARNING : Run accuracy: 0.933520460925
2017-03-12 12:17:00,947 : WARNING : Cat accuracies:
[0.9661708253358925, 0.9195678271308524, 0.8128342245989305]


In [7]:
logging.info("done")
logging.info("accuracies: %s" % accuracies)
logging.info("mean accuracy: %s" % np.mean(accuracies))
logging.info("categories accuracies: \n%s" % cat_accuracies.values)
logging.info("categories accuracies mean: \n%s" % cat_accuracies.mean())
logging.info("done")

2017-03-12 12:17:00,955 : INFO : done
2017-03-12 12:17:00,956 : INFO : accuracies: [0.93280165411312954, 0.93292953168858028, 0.93470231939725212, 0.93396365785197222, 0.93352046092480423]
2017-03-12 12:17:00,956 : INFO : mean accuracy: 0.933583524795
2017-03-12 12:17:00,958 : INFO : categories accuracies: 
[[ 0.95513436  0.89382124  0.90277778]
 [ 0.96089251  0.91296519  0.84385027]
 [ 0.95345489  0.91776711  0.88128342]
 [ 0.97840691  0.89435774  0.80641711]
 [ 0.96617083  0.91956783  0.81283422]]
2017-03-12 12:17:00,959 : INFO : categories accuracies mean: 
eap          0.962812
fuse         0.907696
devstudio    0.849433
dtype: float64
2017-03-12 12:17:00,960 : INFO : done
